# Valuing Caps and Floors with QL Example

We value caps and floors using Black's model and try to replicate a QL example at 
http://gouthamanbalaraman.com/blog/interest-rate-cap-floor-valuation-quantlib-python.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.market.curves import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 07 Sep 2020 at 22:01 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Building a Libor Discount Curve

The example does this using a set of dates and zero ratesm

In [3]:
valuationDate = FinDate(14, 6, 2016)

We have a vector of dates and zero rates

In [13]:
dates = [FinDate(14,6,2016), FinDate(14,9,2016), FinDate(14,12,2016), FinDate(14,6,2017),
         FinDate(14,6,2019), FinDate(14,6,2021), FinDate(15,6,2026), FinDate(16,6,2031),
         FinDate(16,6,2036), FinDate(14,6,2046)]
rates = [0.000000, 0.006616, 0.007049, 0.007795, 0.009599, 0.011203, 0.015068, 0.017583,
         0.018998, 0.020080]

frequencyType = FinFrequencyTypes.ANNUAL
dayCountType = FinDayCountTypes.ACT_ACT_ISDA

In [14]:
discountCurve = FinDiscountCurveZeros(valuationDate, dates, rates, frequencyType, 
                                      dayCountType, FinInterpTypes.LINEAR_ZERO_RATES)

In [15]:
print(discountCurve)

OBJECT TYPE: FinDiscountCurveZeros
VALUATION DATE: TUE 14 JUN 2016
FREQUENCY TYPE: FinFrequencyTypes.ANNUAL
DAY COUNT TYPE: FinDayCountTypes.ACT_ACT_ISDA
INTERP TYPE: FinInterpTypes.LINEAR_ZERO_RATES
DATES: ZERO RATES
TUE 14 JUN 2016:  0.0000000
WED 14 SEP 2016:  0.0066160
WED 14 DEC 2016:  0.0070490
WED 14 JUN 2017:  0.0077950
FRI 14 JUN 2019:  0.0095990
MON 14 JUN 2021:  0.0112030
MON 15 JUN 2026:  0.0150680
MON 16 JUN 2031:  0.0175830
MON 16 JUN 2036:  0.0189980
THU 14 JUN 2046:  0.0200800



In [16]:
startDate = FinDate(14, 6, 2016)
endDate = FinDate(14, 6 , 2026)
calendarType = FinCalendarTypes.US
busDayAdjustType = FinBusDayAdjustTypes.MODIFIED_FOLLOWING
frequencyType = FinFrequencyTypes.QUARTERLY
dateGenRuleType = FinDateGenRuleTypes.FORWARD
lastFixing = 0.0065560
notional = 1000000
dayCountType=FinDayCountTypes.ACT_360
optionType = FinLiborCapFloorTypes.CAP
strikeRate = 0.02

In [8]:
cap = FinLiborCapFloor(startDate, endDate, optionType, strikeRate, lastFixing, 
                       frequencyType,  dayCountType, notional,
                       calendarType, busDayAdjustType, dateGenRuleType)

In [9]:
print(cap)

START DATE: TUE 14 JUN 2016
MATURITY DATE: SUN 14 JUN 2026
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorTypes.CAP
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.ACT_360


## Valuation

### Black's Model

In [10]:
blackVol = 0.547295
model = FinModelBlack(blackVol)

In [11]:
cap.value(valuationDate, discountCurve, model)

54367.93412182139

This is very close to QL which finds 54,369.858

In [12]:
cap.printLeg()

START DATE: TUE 14 JUN 2016
MATURITY DATE: SUN 14 JUN 2026
OPTION TYPE FinLiborCapFloorTypes.CAP
STRIKE (%): 2.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE TUE 14 JUN 2016
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
TUE 14 JUN 2016          -         -            -     1.000000            -            -
WED 14 SEP 2016  0.2555556   0.65560         0.00     0.998340         0.00         0.00
WED 14 DEC 2016  0.2527778   0.73610         0.00     0.996486         0.03         0.03
TUE 14 MAR 2017  0.2500000   0.80274         0.00     0.994490         3.72         3.75
WED 14 JUN 2017  0.2555556   0.87568         0.00     0.992270        25.43        29.18
THU 14 SEP 2017  0.2555556   0.87667         0.00     0.990052        51.40        80.58
THU 14 DEC 2017  0.2527778   0.92092         0.00     0.987752        98.11       178.69
WED 14 MAR 2018  0.2500000   0.96469         0.00     0.985376      

Copyright (c) 2020 Dominic O'Kane

This value is within a few dollars of the Quantlib valuation.